# Desplegar chromaDB

`NOTA:Instalar docker y se recomienda utilizar linux`

In [ ]:
! docker pull chromadb/chroma

## Configuracion de docker

En este caso configuraremos un contenedor que use un token para la autentificacion, para saber sobre mas tipos de autenticacion da puedes ver la documentacion [aqui](https://docs.trychroma.com/usage-guide#authentication) y configuramos un volumen para persistencia de datos

Tambien habilitaremos la opcion de reseteo de la base de datos

In [ ]:
! docker run ^
-p 8000:8000 ^
-e CHROMA_SERVER_AUTH_CREDENTIALS="test-token" ^
-e CHROMA_SERVER_AUTH_CREDENTIALS_PROVIDER="chromadb.auth.token.TokenConfigServerAuthCredentialsProvider" ^
-e CHROMA_SERVER_AUTH_PROVIDER="chromadb.auth.token.TokenAuthServerProvider" ^
-e ALLOW_RESET=TRUE ^
-e IS_PERSISTENT=TRUE ^
-v C:/Users/Dannkol/Documents/FacilPost/docker_prueba/chromadata/:/chroma/chroma ^
chromadb/chroma

In [ ]:
# Verificar que el contenedor esta corriendo

! curl  -X GET \
  'http://localhost:8000/api/v1/tenants/default_tenant' \
  --header 'Accept: */*' \
  --header 'User-Agent: Thunder Client (https://www.thunderclient.com)' \
  --header 'Authorization: Bearer test-token'

# Conectamos el cliente

Dependiendo de nuestra configuracion la conexion al backend es diferente, en este caso se realiza por medio de un token y una api, entonces podremos hacer peticiones pormedio de la api mandando un header de `Authorization`

In [63]:
import chromadb
from chromadb.config import Settings

client = chromadb.HttpClient(
    settings=Settings(
        chroma_client_auth_provider="chromadb.auth.token.TokenAuthClientProvider",
        chroma_client_auth_credentials="test-token", 
        allow_reset=True)
        )

client.heartbeat()  

client.get_version()  

client.list_collections()  

[]

# Integracion con langchain

In [56]:
# Creamos una nueva colección
colección = client.create_collection("test-collection")

# Verificamos que la colección se haya creado
client.list_collections()

[Collection(name=test-collection)]

In [57]:
# Creamos un nuevo documento
descripcionSrAnillos = "El señor de los anillos: La comunidad del anillo es una película de fantasía y aventuras de 2001 dirigida por Peter Jackson y basada en el primer volumen de la novela homónima de J. R. R. Tolkien. Es la primera entrega de la trilogía cinematográfica El Señor de los Anillos, producida por New Line Cinema y protagonizada por Elijah Wood, Ian McKellen, Liv Tyler y Viggo Mortensen, entre otros. La historia se centra en el joven hobbit Frodo Bolsón, quien emprende un viaje para destruir el Anillo Único y poner fin al poder maléfico del Señor Oscuro Sauron. Se trata de la primera adaptación de la novela de Tolkien al cine, y fue escrita por Fran Walsh, Philippa Boyens, Stephen Sinclair y el propio Jackson."

descripcionBatman = "Batman es una película de superhéroes británico-estadounidense de 1989 dirigida por Tim Burton y producida por Jon Peters y Peter Guber, con un guion de Sam Hamm y Warren Skaaren. Está basada en el personaje homónimo de DC Comics creado por Bob Kane e interpretado por Michael Keaton. La película está protagonizada por Jack Nicholson como el Joker, Kim Basinger como Vicki Vale, Robert Wuhl como Alexander Knox y Jack Palance como Carl Grissom. La película se centra en el origen del superhéroe Batman y su lucha contra el Joker, un villano que planea dominar la ciudad de Gotham con un misterioso gas que causa risa."

descripcionHarryPotter = "Harry Potter y la piedra filosofal es una película de fantasía y aventuras basada en el libro homónimo de J. K. Rowling, dirigida por Chris Columbus y estrenada en 2001. Es la primera entrega de la serie de películas de Harry Potter. La película fue escrita por Steve Kloves y producida por David Heyman. La historia sigue a Harry Potter, un niño que descubre que es un mago y es enviado a estudiar a la Escuela Hogwarts de Magia y Hechicería. Allí se entera de que el mago tenebroso Lord Voldemort, que asesinó a sus padres, está vivo y planea volver al poder. La película fue distribuida por Warner Bros. Pictures y protagonizada por Daniel Radcliffe como Harry Potter, con Rupert Grint como Ron Weasley y Emma Watson como Hermione Granger."

descripcionMatrix = "Matrix es una película de ciencia ficción y acción estadounidense de 1999 escrita y dirigida por las hermanas Wachowski y protagonizada por Keanu Reeves, Laurence Fishburne, Carrie-Anne Moss y Hugo Weaving. Está ambientada en un futuro distópico donde la realidad percibida por la mayoría de los humanos es en realidad una simulación virtual llamada «Matrix», creada por máquinas inteligentes para esclavizar a la raza humana, mientras que sus cuerpos caloríficos son usados como fuente de energía. Un pirata informático llamado Neo descubre esta verdad y se ve arrastrado a una rebelión contra las máquinas, que involucra a otras personas que han sido liberadas de la Matrix."


# Agregar documentos a la colección
# Les agregamos un metadato de genero
colección.add(
    documents = [descripcionSrAnillos, descripcionBatman, descripcionHarryPotter, descripcionMatrix],
    metadatas = [{"genero": "fantasia"},{"genero": "accion"},{'genero':'fantasia'},{'genero':'accion'}],
    ids = ["id1", "id2", "id3", "id4"]
)

In [58]:
# Verificamos que los documentos se hayan agregado
colección.get()

{'ids': ['id1', 'id2', 'id3', 'id4'],
 'embeddings': None,
 'metadatas': [{'genero': 'fantasia'},
  {'genero': 'accion'},
  {'genero': 'fantasia'},
  {'genero': 'accion'}],
 'documents': ['El señor de los anillos: La comunidad del anillo es una película de fantasía y aventuras de 2001 dirigida por Peter Jackson y basada en el primer volumen de la novela homónima de J. R. R. Tolkien. Es la primera entrega de la trilogía cinematográfica El Señor de los Anillos, producida por New Line Cinema y protagonizada por Elijah Wood, Ian McKellen, Liv Tyler y Viggo Mortensen, entre otros. La historia se centra en el joven hobbit Frodo Bolsón, quien emprende un viaje para destruir el Anillo Único y poner fin al poder maléfico del Señor Oscuro Sauron. Se trata de la primera adaptación de la novela de Tolkien al cine, y fue escrita por Fran Walsh, Philippa Boyens, Stephen Sinclair y el propio Jackson.',
  'Batman es una película de superhéroes británico-estadounidense de 1989 dirigida por Tim Burton y

In [59]:
# Importamos los paquetes necesarios
from langchain.vectorstores import Chroma
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings

In [60]:
# Definimos la función de embedding
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
# Instanciamos el nuesta base de datos vectorial
db = Chroma(
    client=client,
    collection_name="test-collection",
    embedding_function=embedding_function,
)

In [61]:
# Verificamos que la integracion este bien
db.get()

{'ids': ['id1', 'id2', 'id3', 'id4'],
 'embeddings': None,
 'metadatas': [{'genero': 'fantasia'},
  {'genero': 'accion'},
  {'genero': 'fantasia'},
  {'genero': 'accion'}],
 'documents': ['El señor de los anillos: La comunidad del anillo es una película de fantasía y aventuras de 2001 dirigida por Peter Jackson y basada en el primer volumen de la novela homónima de J. R. R. Tolkien. Es la primera entrega de la trilogía cinematográfica El Señor de los Anillos, producida por New Line Cinema y protagonizada por Elijah Wood, Ian McKellen, Liv Tyler y Viggo Mortensen, entre otros. La historia se centra en el joven hobbit Frodo Bolsón, quien emprende un viaje para destruir el Anillo Único y poner fin al poder maléfico del Señor Oscuro Sauron. Se trata de la primera adaptación de la novela de Tolkien al cine, y fue escrita por Fran Walsh, Philippa Boyens, Stephen Sinclair y el propio Jackson.',
  'Batman es una película de superhéroes británico-estadounidense de 1989 dirigida por Tim Burton y

# reset

In [62]:
client.reset()

True